In [1]:
#----------------------------------------------------------
# Notebook to create database "immobilier" from excel file
#
# Upgrades :
# => code SQLalchemy class to automate table creation
# => automate key generation between tables
#----------------------------------------------------------

from My_db_connector import My_db_connector
import sqlite3 as sql
import pandas as pd
import numpy as np

# load data and drop duplicates
df = pd.read_excel("immobilier.xlsx")
df = df.drop_duplicates()

# formatting attributs
df.columns = [i.replace(' ', '_').lower() for i in df.columns]

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
# dataframe summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34169 entries, 0 to 34168
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_mutation              34169 non-null  datetime64[ns]
 1   nature_mutation            34169 non-null  object        
 2   valeur_fonciere            34151 non-null  float64       
 3   no_voie                    34036 non-null  float64       
 4   b/t/q                      2174 non-null   object        
 5   code_type_de_voie          34169 non-null  int64         
 6   type_de_voie               33229 non-null  object        
 7   code_voie                  34169 non-null  object        
 8   voie                       34169 non-null  object        
 9   code_id_commune            34169 non-null  int64         
 10  code_postal                34168 non-null  float64       
 11  commune                    34169 non-null  object        
 12  code

In [3]:
# select all attributs needed in logement_table from df.index
logement_attr = list(df.columns[[3, 6, 7, 8, 11]])
print("table 'logement' :", "\n", logement_attr, "\n")

# select all attributs needed in vente_table from df.index
vente_attr = list(df.columns[[0, 2, 18, 23, 21, 9, 11, 12]])
print("table 'vente' :", "\n", vente_attr)

NameError: name 'vente_attr' is not defined

In [ ]:
# create logement dataframe and drop duplicates to have unique "logements" in this table
df_logement = df.loc[:, logement_attr]
df_logement = df_logement.drop_duplicates().reset_index()
df_logement

In [ ]:
# create vente dataframe and add vente_id

df_vente = df.loc[:, vente_attr]

# add foreign_key logement_id to df_vente
for i in range(len(df_vente)):
    result, = np.where(df_logement["commune"] == df_vente["commune"].loc[i])
    df_vente.loc[i, "logement_id"] = result[0]
del i, result

df_vente

In [ ]:
# add vente_id and logement_id to vente_attr
df_vente["vente_id"] = df_vente.index
vente_attr.append("vente_id")
vente_attr.append("logement_id")

# add logement_id to logement_attr
df_logement["logement_id"] = df_logement.index
logement_attr.append("logement_id")

print("table 'vente' :", "\n", vente_attr, "\n")
print("table 'logement' :", "\n", logement_attr)

In [ ]:
# connection to database
connector = My_db_connector("immobilier")

# populate both tables with to_sql() method
connector.df_to_sql(df_vente, "vente")
connector.df_to_sql(df_logement, "logement")

In [ ]:
# display all table from database
tables = ["vente", "logement"]
query = "SELECT * FROM {}"

for table in tables:
    result = connector.db_execute_query(query.format(table))
    display(result)
    
print("Database 'immobilier' ready to run")

In [ ]:
# close connection 
connector.db_close_connection()